In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from keras.utils import image_dataset_from_directory

In [4]:
train_ds = image_dataset_from_directory(directory='dataset_split/train',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    format='tf',
    verbose=True,)

Found 926 files belonging to 9 classes.


In [5]:
test_ds = image_dataset_from_directory(directory='dataset_split/test',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=False,
    seed=42,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    format='tf',
    verbose=True,)

Found 237 files belonging to 9 classes.


In [6]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

## Looking over labels

In [7]:
print(train_ds.class_names)


['buoy', 'cruise_ship', 'ferry_boat', 'freight_boat', 'gondola', 'inflatable_boat', 'kayak', 'paper_boat', 'sailboat']


In [8]:
all_labels = []

for _, labels in train_ds:
    all_labels.extend(labels.numpy())

print(np.unique(all_labels))


[0 1 2 3 4 5 6 7 8]


In [12]:
for x,y in train_ds:
    print(x.shape)
    break

(32, 256, 256, 3)


## CNN - scratch

In [7]:
from tensorflow.keras import layers, Sequential

In [14]:
model = Sequential([
    
    layers.Input(shape=(256, 256, 3)),

    layers.Rescaling(scale=1./255),

    layers.Conv2D(filters = 16, kernel_size=(3,3), activation='relu', padding="same") , 
    layers.MaxPool2D(pool_size=(2,2)),

    layers.Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding="same") , 
    layers.MaxPool2D(pool_size=(2,2)),

    layers.Flatten(),

    layers.Dense(units = 64, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(units = 9, activation = 'softmax'),
    
])

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape              ┃        Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)              │ (None, 256, 256, 3)       │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d (Conv2D)                    │ (None, 256, 256, 16)      │            448 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d (MaxPooling2D)       │ (None, 128, 128, 16)      │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d_1 (Conv2D)                  │ (None, 128, 128, 32)      │          4,640 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d_1 (MaxPooling2D)     │ (None, 64, 64, 32)        │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ flatten (Flatten)                  │ (None, 131072)            │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense (Dense)                      │ (None, 64)                │      8,388,672 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dropout (Dropout)                  │ (None, 64)                │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_1 (Dense)                    │ (None, 9)                 │            585 │
└────────────────────────────────────┴───────────────────────────┴────────────────┘

 Total params: 8,394,345 (32.02 MB)

 Trainable params: 8,394,345 (32.02 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(optimizer = 'adam',
             loss='sparse_categorical_crossentropy',
             metrics = ['accuracy'])

## Fitting on train data

In [17]:
model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 210ms/step - accuracy: 0.2073 - loss: 2.9142 - val_accuracy: 0.2532 - val_loss: 1.8957
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 206ms/step - accuracy: 0.3521 - loss: 1.7539 - val_accuracy: 0.3797 - val_loss: 1.6978
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 205ms/step - accuracy: 0.5259 - loss: 1.4194 - val_accuracy: 0.4177 - val_loss: 1.6097
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 206ms/step - accuracy: 0.6609 - loss: 1.0713 - val_accuracy: 0.4937 - val_loss: 1.5587
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 222ms/step - accuracy: 0.7559 - loss: 0.7762 - val_accuracy: 0.5148 - val_loss: 1.6666
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step - accuracy: 0.8521 - loss: 0.5187 - val_accuracy: 0.5612 - val_loss: 1.7045
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 205ms/step - accuracy: 0.9017 - loss: 0.3316 - val_accuracy: 0.5190 - val_loss: 1.9216
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - accuracy: 0.9417 - loss: 0.2358 - val_accuracy: 0

## doing as per tasks given...

to use categorical_crossentropy and their specific layer architechture

In [4]:
train_ds = image_dataset_from_directory(
    "dataset_split/train",
    image_size=(256,256),
    batch_size=32,
    label_mode='categorical',  # 1 hot encoded
    shuffle=True,
    seed=43
)

test_ds = image_dataset_from_directory(
    "dataset_split/test",
    image_size=(256,256),
    batch_size=32,
    label_mode='categorical',  
    shuffle=False
)


Found 926 files belonging to 9 classes.
Found 237 files belonging to 9 classes.


In [5]:
for x,y in train_ds:
    print(x.shape)
    break

(32, 256, 256, 3)


### Model2 build
according the task assigned

In [25]:
model2 = Sequential([
    
    layers.Input(shape=(256, 256, 3)),

    layers.Rescaling(scale=1./255),

    layers.Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding="same") , 
    layers.MaxPool2D(pool_size=(2,2)),

    layers.Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding="same") , 
    layers.MaxPool2D(pool_size=(2,2)),

    layers.GlobalAveragePooling2D(),
    
    layers.Dense(units = 128, activation = 'relu'),
    layers.Dense(units = 128, activation = 'relu'),
    layers.Dense(units = 9, activation = 'softmax'),
    
])

In [24]:
model2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape              ┃        Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)            │ (None, 256, 256, 3)       │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d_2 (Conv2D)                  │ (None, 256, 256, 32)      │            896 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d_2 (MaxPooling2D)     │ (None, 128, 128, 32)      │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d_3 (Conv2D)                  │ (None, 128, 128, 32)      │          9,248 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d_3 (MaxPooling2D)     │ (None, 64, 64, 32)        │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ global_average_pooling2d           │ (None, 32)                │              0 │
│ (GlobalAveragePooling2D)           │                           │                │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_2 (Dense)                    │ (None, 128)               │          4,224 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_3 (Dense)                    │ (None, 128)               │         16,512 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_4 (Dense)                    │ (None, 9)                 │          1,161 │
└────────────────────────────────────┴───────────────────────────┴────────────────┘

 Total params: 32,041 (125.16 KB)

 Trainable params: 32,041 (125.16 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model2.compile(optimizer = 'adam',
              loss='categorical_crossentropy',
              metrics=[ 'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')])

In [31]:
model2.fit(train_ds, epochs=20, validation_data=test_ds)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 294ms/step - accuracy: 0.3240 - loss: 1.9261 - precision: 0.4595 - recall: 0.0184 - val_accuracy: 0.3291 - val_loss: 1.8223 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 306ms/step - accuracy: 0.3359 - loss: 1.8044 - precision: 1.0000 - recall: 0.0032 - val_accuracy: 0.3291 - val_loss: 1.8051 - val_precision: 1.0000 - val_recall: 0.0169
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 289ms/step - accuracy: 0.3359 - loss: 1.7994 - precision: 0.5833 - recall: 0.0076 - val_accuracy: 0.3291 - val_loss: 1.8019 - val_precision: 0.8333 - val_recall: 0.0422
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 288ms/step - accuracy: 0.3359 - loss: 1.7997 - precision: 0.6000 - recall: 0.0162 - val_accuracy: 0.3291 - val_loss: 1.8080 - val_precision: 0.7826 - val_recall: 0.0759
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 306ms/step - accuracy: 0.3359 - loss: 1.7695 - precision: 0.7692 - recall: 0.0216 - val_accuracy: 0.3291 - v

## Imrproving the model myself then evaluation...

#### model2 is underfit
so for imprving it, will do data augmentation, batch normalization, dropout layers...

ImageGenerator of keras will be deprecated and works slow and doesn't work with dataset pipelines so will use Sequential

In [8]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [9]:
model2_imp = Sequential([
    
    layers.Input(shape=(256, 256, 3)),

    data_augmentation, # augmentation before scaling as it adds more images...
    
    layers.Rescaling(scale=1./255),

    layers.Conv2D(16, (3,3), padding='same'),
    layers.BatchNormalization(), # activation before normalization isn't good so will do batch norm, then activation relu
    layers.Activation('relu'),   
    layers.MaxPooling2D(),
 
    layers.Conv2D(32, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(),

    layers.GlobalAveragePooling2D(),
    
    layers.Dense(units = 128, activation = 'relu'),
    layers.Dropout(0.4),  # dropout layer 1 
    
    layers.Dense(units = 128, activation = 'relu'),
    layers.Dropout(0.3),  # dropout layer 2
    
    layers.Dense(units = 9, activation = 'softmax'),
    
])

In [10]:
model2_imp.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape              ┃        Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)            │ (None, 256, 256, 3)       │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ rescaling (Rescaling)              │ (None, 256, 256, 3)       │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d (Conv2D)                    │ (None, 256, 256, 16)      │            448 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ batch_normalization                │ (None, 256, 256, 16)      │             64 │
│ (BatchNormalization)               │                           │                │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ activation (Activation)            │ (None, 256, 256, 16)      │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d (MaxPooling2D)       │ (None, 128, 128, 16)      │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d_1 (Conv2D)                  │ (None, 128, 128, 32)      │          4,640 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ batch_normalization_1              │ (None, 128, 128, 32)      │            128 │
│ (BatchNormalization)               │                           │                │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ activation_1 (Activation)          │ (None, 128, 128, 32)      │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d_1 (MaxPooling2D)     │ (None, 64, 64, 32)        │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ conv2d_2 (Conv2D)                  │ (None, 64, 64, 64)        │         18,496 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ batch_normalization_2              │ (None, 64, 64, 64)        │            256 │
│ (BatchNormalization)               │                           │                │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ activation_2 (Activation)          │ (None, 64, 64, 64)        │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ max_pooling2d_2 (MaxPooling2D)     │ (None, 32, 32, 64)        │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ global_average_pooling2d           │ (None, 64)                │              0 │
│ (GlobalAveragePooling2D)           │                           │                │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense (Dense)                      │ (None, 128)               │          8,320 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dropout (Dropout)                  │ (None, 128)               │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_1 (Dense)                    │ (None, 128)               │         16,512 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dropout_1 (Dropout)                │ (None, 128)               │              0 │
├────────────────────────────────────┼───────────────────────────┼────────────────┤
│ dense_2 (Dense)                    │ (None, 9)                 │          1,161 │
└────────────────────────────────────┴───────────────────────────┴────────────────┘

 Total params: 50,025 (195.41 KB)

 Trainable params: 49,801 (194.54 KB)

 Non-trainable params: 224 (896.00 B)

In [11]:
model2_imp.compile(optimizer = 'adam',
              loss='categorical_crossentropy',
              metrics=[ 'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')])

In [ ]:
history2 = model2_imp.fit(train_ds, epochs=20, validation_data=test_ds)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 23s 713ms/step - accuracy: 0.2991 - loss: 1.9011 - precision: 0.4930 - recall: 0.0378 - val_accuracy: 0.3291 - val_loss: 2.0140 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 683ms/step - accuracy: 0.3823 - loss: 1.7010 - precision: 0.5986 - recall: 0.0950 - val_accuracy: 0.3291 - val_loss: 1.9665 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 677ms/step - accuracy: 0.4104 - loss: 1.6875 - precision: 0.5251 - recall: 0.1242 - val_accuracy: 0.3291 - val_loss: 1.9230 - val_precision: 0.8000 - val_recall: 0.0169
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 685ms/step - accuracy: 0.4114 - loss: 1.6557 - precision: 0.5714 - recall: 0.1339 - val_accuracy: 0.3291 - val_loss: 1.9426 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/20
23/29 ━━━━━━━━━━━━━━━━━━━━ 4s 683ms/step - accuracy: 0.4420 - loss: 1.5969 - precision: 0.6164 - recall: 0.1505 